In [1]:
import numpy as np
import pandas as pd
from utils.diff import diff

In [2]:
# net income before extraordinary items 特别项目前收入
# extraordinary items可以认为是营业外净收入
# 净利润 + 营业外支出 - 营业外收入
# 或：营业利润 - 所得税

In [3]:
import pandas as pd
import numpy as np
from utils.diff import diff  # 确保 diff 函数正确导入

# 读取资产负债表
df1 = pd.read_csv('季_资产负债表.csv', usecols=['证券代码', '会计期间', '资产总计', '所有者权益合计'], low_memory=False)

# 确保证券代码为整数类型
df1['证券代码'] = pd.to_numeric(df1['证券代码'], errors='coerce').fillna(0).astype(int)

# 读取利润表
df2 = pd.read_csv('季_利润表.csv', usecols=['证券代码', '会计期间', '营业利润', '减：所得税费用'])

# 确保证券代码为整数类型
df2['证券代码'] = pd.to_numeric(df2['证券代码'], errors='coerce').fillna(0).astype(int)

# 合并数据
df3 = pd.merge(df1, df2, on=['证券代码', '会计期间'], how='left')

# 计算营业利润和所得税费用的差异
df3 = diff(df3, col=['营业利润', '减：所得税费用'])
df3['特别项目前收入'] = df3['营业利润'] - df3['减：所得税费用']

# 计算 roaq
df3['roaq'] = df3['特别项目前收入'] / df3.groupby('证券代码').shift(1)['资产总计'].replace(0, np.nan)

# 计算 roavol
df3['平均资产'] = df3[['证券代码', '资产总计']].groupby('证券代码').rolling(window=16).mean().reset_index(level=0, drop=True)
df3['特别项目前收入标准差'] = df3[['证券代码', '特别项目前收入']].groupby('证券代码').rolling(window=16).std().reset_index(level=0, drop=True)
df3['roavol'] = df3['特别项目前收入标准差'] / df3['平均资产'].replace(0, np.nan)

# 计算 roeq
df3['roeq'] = df3['特别项目前收入'] / df3.groupby('证券代码').shift(1)['所有者权益合计'].replace(0, np.nan)

# 计算 operprof
df3['operprof'] = df3['营业利润'] / df3.groupby('证券代码').shift(1)['所有者权益合计'].replace(0, np.nan)

# 计算 chpm
df3['chpm'] = (df3['特别项目前收入'] - df3.groupby('证券代码').shift(1)['特别项目前收入']) / df3['资产总计'].replace(0, np.nan)

# 读取公司文件
df4 = pd.read_csv('公司文件.csv', usecols=['证券代码', '行业代码C'])

# 确保证券代码为整数类型
df4['证券代码'] = pd.to_numeric(df4['证券代码'], errors='coerce').fillna(0).astype(int)

# 合并数据
df5 = pd.merge(df3, df4, on=['证券代码'])

# 计算行业调整后的 chpm
df6 = df5[['行业代码C', '会计期间', 'chpm']].groupby(['行业代码C', '会计期间']).mean().reset_index()
df6.rename(columns={'chpm': 'chpm_i'}, inplace=True)

# 合并行业平均
df7 = pd.merge(df5, df6, on=['行业代码C', '会计期间'], how='left')
df7['chpm_ia'] = df7['chpm'] - df7['chpm_i']

# 选择最终列并导出
df8 = df7[['证券代码', '会计期间', 'chpm', 'chpm_ia', 'operprof', 'roaq', 'roavol', 'roeq']]
df8.to_csv('季_19_20_49_69_70_71.csv', index=False, encoding='utf-8-sig')

# 输出结果 DataFrame
print(df8)

          证券代码        会计期间  chpm  chpm_ia  operprof  roaq  roavol  roeq
0            1  2000-03-31   NaN      NaN       NaN   NaN     NaN   NaN
1            1  2000-06-30   NaN      NaN       NaN   NaN     NaN   NaN
2            1  2000-09-30   NaN      NaN       NaN   NaN     NaN   NaN
3            1  2000-12-31   NaN      NaN       NaN   NaN     NaN   NaN
4            1  2001-03-31   NaN      NaN       NaN   NaN     NaN   NaN
...        ...         ...   ...      ...       ...   ...     ...   ...
571126  920118  2023-09-30   NaN      NaN       NaN   NaN     NaN   NaN
571127  920118  2023-12-31   NaN      NaN       NaN   NaN     NaN   NaN
571128  920118  2024-03-31   NaN      NaN       NaN   NaN     NaN   NaN
571129  920118  2024-06-30   NaN      NaN       NaN   NaN     NaN   NaN
571130  920118  2024-09-30   NaN      NaN       NaN   NaN     NaN   NaN

[571131 rows x 8 columns]
